# CMSE 201 First Project Checkpoint

### &#9989; Aaron Ko, Sit Soe, Sean O'Dwyer, Abdulla Alblooshi
### &#9989; Section_001

## Background and Motivation

PLACEHOLDER

## Methodology

We will analyze our data using pandas, numpy, and python functions.

### First, lets import everything we'll need:

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split

### Cleaning up the dataset:

In [28]:
data = pd.read_csv("FirstYearGPA.csv")
data.head()

,Unnamed: 0,GPA,HSGPA,SATV,SATM,Male,HU,SS,FirstGen,White,CollegeBound
0,1,3.06,3.83,680,770,1,3.0,9.0,1,1,1
1,2,4.15,4.00,740,720,0,9.0,3.0,0,1,1
2,3,3.41,3.70,640,570,0,16.0,13.0,0,0,1
3,4,3.21,3.51,740,700,0,22.0,0.0,0,1,1
4,5,3.48,3.83,610,610,0,30.5,1.5,0,1,1


### Combine the SATV and SATM data in to one column that represents the total SAT score

In [29]:
# combine the two columns
data["SATV"] = data["SATV"]+data["SATM"]
# drop the SATM column
data = data.drop("SATM", axis = 1)
# rename the SATV (combined score) column to SAT Score
data.rename(columns = {"SATV":"SAT Score"}, inplace = True)

### Drop NA and unnecessary columns in the data:

In [30]:
data = data.dropna()
data.head()

,Unnamed: 0,GPA,HSGPA,SAT Score,Male,HU,SS,FirstGen,White,CollegeBound
0,1,3.06,3.83,1450,1,3.0,9.0,1,1,1
1,2,4.15,4.00,1460,0,9.0,3.0,0,1,1
2,3,3.41,3.70,1210,0,16.0,13.0,0,0,1
3,4,3.21,3.51,1440,0,22.0,0.0,0,1,1
4,5,3.48,3.83,1220,0,30.5,1.5,0,1,1
